Question 1:

In [5]:
import numpy as np
import pandas as pd
from scipy.io import arff
from scipy.stats import chi2

df = pd.read_csv("glass.csv")
# df = pd.DataFrame(data[0])
# df = df.sample(n=5000, random_state=42)

# print(df.shape)

num_cols = df.select_dtypes(include=[np.number]).columns
data = df[num_cols]

import numpy as np
from scipy.spatial.distance import cdist

def LOF(X, k):
    # Compute pairwise Euclidean distances between points
    D = cdist(X, X)
    
    # Get indices of k nearest neighbors for each point
    knn_indices = np.argsort(D, axis=1)[:, 1:k+1]
    
    RD = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        xi = X[i]
        x_knn_indices = knn_indices[i]
        for j in x_knn_indices:
            xj = X[j]
            dist = np.linalg.norm(xi - xj)
            RD[i] += max(dist, np.max(D[j, x_knn_indices]))
        RD[i] /= k
    
    # Compute Local Reachability Density (LRD) for each point
    LRD = 1 / (np.sum(RD[knn_indices], axis=1) / k)
    
    LOF = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        xi = X[i]
        x_knn_indices = knn_indices[i]
        LOF[i] = np.sum(LRD[x_knn_indices]) / (LRD[i] * k)
        
    return LOF


def MahalonobisDist(x, data, cov=None):
    x_mean = x - np.mean(data, axis=0)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = np.linalg.inv(cov)
    left = np.dot(x_mean, inv_covmat)
    mahalonobis = np.dot(left, x_mean.T)
    
    return mahalonobis.diagonal()

df["mahalonobis"] = MahalonobisDist(df[num_cols], data)
df['p'] = 1 - chi2.cdf(df['mahalonobis'], 28)

# print(df.head())
print(df[df['p'] < 0.001])

num_cols = df.select_dtypes(include=[np.number]).columns

# Compute LOF scores
X = df[num_cols].values
k = 20
lof_scores = LOF(X, k)

# Add LOF scores to dataframe
df['lof_score'] = lof_scores

outliers = df[df['lof_score'] > 1.5]
print("Number of outliers detected:", outliers.shape[0])
print(outliers)


def otsu_threshold(data):
    hist, bins = np.histogram(data, bins=256, range=(0, 255))
    hist_norm = hist.astype(np.float32) / hist.sum()
    bin_centers = (bins[:-1] + bins[1:]) / 2.

    # Computing inter-class variance for all possible thresholds
    weight_background = np.cumsum(hist_norm)
    mean_background = np.cumsum(hist_norm * bin_centers) / weight_background
    mean_foreground = np.cumsum((hist_norm * bin_centers)[::-1])[::-1] / (1 - weight_background)

    variance_between_classes = weight_background[:-1] * (1 - weight_background[:-1]) * ((mean_background[:-1] - mean_foreground[1:]) ** 2)

    # Finding the threshold with maximum inter-class variance
    idx_max = np.argmax(variance_between_classes)
    threshold = bin_centers[:-1][idx_max]

    return threshold


# Select a single feature for Otsu's thresholding
feature = "RI"
x = df[feature]

thresh = otsu_threshold(x)
binary = x > thresh

print("Otsu's threshold:", thresh)
print("Number of pixels above the threshold:", np.sum(binary))


          RI     Na    Mg    Al     Si     K     Ca    Ba    Fe  Type  \
106  1.53125  10.73  0.00  2.10  69.81  0.58  13.30  3.15  0.28     2   
149  1.51643  12.16  3.52  1.35  72.89  0.57   8.53  0.00  0.00     3   
171  1.51316  13.02  0.00  3.04  70.48  6.21   6.96  0.00  0.00     5   
172  1.51321  13.00  0.00  3.02  70.70  6.21   6.93  0.00  0.00     5   
184  1.51115  17.38  0.00  0.34  75.41  0.00   6.65  0.00  0.00     6   

     mahalonobis             p  
106    87.095612  5.599748e-08  
149    63.641306  1.367545e-04  
171    87.576067  4.720268e-08  
172    87.788275  4.376657e-08  
184    65.597294  7.489330e-05  
Number of outliers detected: 5
          RI     Na    Mg    Al     Si     K     Ca    Ba    Fe  Type  \
106  1.53125  10.73  0.00  2.10  69.81  0.58  13.30  3.15  0.28     2   
149  1.51643  12.16  3.52  1.35  72.89  0.57   8.53  0.00  0.00     3   
171  1.51316  13.02  0.00  3.04  70.48  6.21   6.96  0.00  0.00     5   
172  1.51321  13.00  0.00  3.02  70.70  

<ipython-input-5-e0db1f34b115>:85: RuntimeWarning: invalid value encountered in true_divide
  mean_background = np.cumsum(hist_norm * bin_centers) / weight_background
<ipython-input-5-e0db1f34b115>:86: RuntimeWarning: divide by zero encountered in true_divide
  mean_foreground = np.cumsum((hist_norm * bin_centers)[::-1])[::-1] / (1 - weight_background)
<ipython-input-5-e0db1f34b115>:86: RuntimeWarning: invalid value encountered in true_divide
  mean_foreground = np.cumsum((hist_norm * bin_centers)[::-1])[::-1] / (1 - weight_background)


Question 2:

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict(X, w):
    z = np.clip(X.dot(w), -500, 500) # limit z to avoid overflow
    return sigmoid(z)

def train(X, y, lr=0.01, epochs=1000):
    w = np.zeros(X.shape[1])
    for epoch in range(epochs):
        y_pred = predict(X, w)
        error = y - y_pred
        gradient = X.T.dot(error)
        w += lr * gradient
    return w

def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def main():
    # Load data
    df = pd.read_csv("Heart.csv", header=0)

    df.dropna(inplace=True)
    X = df.drop('AHD', axis=1)
    y = df['AHD']

    # Convert categorical variables to numerical using one-hot encoding
    df = pd.get_dummies(df, columns=["ChestPain", "Thal"])

    X = df.drop(["AHD"], axis=1)
    y = df["AHD"]

    # Convert categorical target variable to numerical
    y = pd.get_dummies(y, drop_first=True).values.ravel()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

    from sklearn.decomposition import PCA
    # Apply PCA to reduce the feature dimension
    pca = PCA(n_components=3)
    X_train_transformed_pca = pca.fit_transform(X_train)
    X_test_transformed_pca = pca.transform(X_test)

    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    lda = LinearDiscriminantAnalysis(n_components=1)
    X_train_transformed_fda = lda.fit_transform(X_train_transformed_pca, y_train)
    X_test_transformed_fda = lda.transform(X_test_transformed_pca)

    # Train logistic regression model without FDA+PCA
    w_before = train(X_train, y_train)
    y_pred_before = (predict(X_test, w_before) > 0.5).astype(int)
    accuracy_before = accuracy(y_test.astype(int), y_pred_before)

    print(f"Accuracy before: {accuracy_before}")

    # Train logistic regression model after FDA+PCA
    w_after_fda_pca = train(X_train_transformed_fda, y_train)
    y_pred_after_fda_pca = (predict(X_test_transformed_fda, w_after_fda_pca) > 0.5).astype(int)

    accuracy_after_fda_pca = accuracy(y_test.astype(int), y_pred_after_fda_pca)

    print(f"Accuracy after PCA+FDA: {accuracy_after_fda_pca}")

    w_after_fda = train(X_train_transformed_fda, y_train)

    y_pred_after_fda = (predict(X_test_transformed_fda, w_after_fda) > 0.5).astype(int)

    accuracy_after_fda = accuracy(y_test.astype(int), y_pred_after_fda)

    print(f"Accuracy after FDA: {accuracy_after_fda}")


main()


Accuracy before: 0.55
Accuracy after PCA+FDA: 0.7833333333333333
Accuracy after FDA: 0.7833333333333333


Question 3:

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics

df = pd.read_csv("RealEstate.csv", header=0)
df.drop('No', axis=1, inplace=True)

df = df.dropna() #It allows to neglect the null values in the data set

df.insert(0, 'Ones', 1)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values.reshape(-1, 1)

Transposed_X = np.transpose(X)

X_Matrix_Multiplied = Transposed_X.dot(X)

Inv_X = np.linalg.inv(X_Matrix_Multiplied)

X_y_dotProduct = Transposed_X.dot(y)

theta = Inv_X.dot(X_y_dotProduct)

Intercept = theta[0]
slope = theta[1]

y_pred = X.dot(theta)
rmse = np.sqrt(np.mean((y_pred - y) ** 2))
rss = np.sum((y_pred - y) ** 2)
tss = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - (rss / tss)

print("RMSE:", rmse)
print("RSS:", rss)
print("R-squared:", r2)


RMSE: 8.782466464603045
RSS: 31932.530921577123
R-squared: 0.582370447272281


To derive the normal equation, we start by assuming that the linear regression model has the form y = Xθ + ε, where y is the dependent variable, X is the independent variable, θ is a vector of parameters, and ε is the error term.

θ = (X^TX)^-1* (X^T*y)

The normal equation, which is this expression's counterpart, provides us with the θ parameters' values.

This formula is used in the given code to determine the intercept and slope of the linear regression model, which are the values of theta. Theta vector's first element contains the intercept, while its second element contains the slope.

The code then computes the RMSE, RSS, and R-squared metrics to assess how well the linear regression model performed on the training set of data.


**Limitations of this approach:**
We need to compute (X^T*X)-1 and it is very slow if the value of n is very large. 

Question 4:

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

class LDA:
    def __init__(self, n_components=None):
        self.n_components = n_components
        self.w = None
        
    def fit(self, X, y):
        classes = np.unique(y)
        means = [np.mean(X[y == c], axis=0) for c in classes]
        
        Sw = np.zeros((X.shape[1], X.shape[1]))
        for c in classes:
            Xc = X[y == c]
            mc = means[c]
            for x in Xc:
                x = x.reshape(-1, 1)
                mc = mc.reshape(-1, 1)
                Sw += (x - mc) @ (x - mc).T
        
        Sb = np.zeros((X.shape[1], X.shape[1]))
        for c in classes:
            Xc = X[y == c]
            mc = means[c]
            Nc = Xc.shape[0]
            mc = mc.reshape(-1, 1)
            m = np.mean(X, axis=0).reshape(-1, 1)
            Sb += Nc * (mc - m) @ (mc - m).T
        
        A = np.linalg.inv(Sw) @ Sb
        eigenvalues, eigenvectors = np.linalg.eig(A)
        
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]

        if self.n_components is not None:
            eigenvectors = eigenvectors[:, :self.n_components]
        
        self.w = eigenvectors
    
    def transform(self, X):
        return X @ self.w

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

lda = LDA(n_components=2)
lda.fit(X_train, y_train)
X_train_lda = lda.transform(X_train)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_lda, y_train)

X_test_lda = lda.transform(X_test)
y_pred = knn.predict(X_test_lda)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f}")


Accuracy: 0.91


Question 5:

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost(X, y, theta):
    m = len(y)
    h = sigmoid(np.dot(X, theta))
    h = np.clip(h, 0.000001, 0.999999)  
    J = (-1/m) * np.sum(y * np.log(h) + (1-y) * np.log(1-h))
    return J

def gradient_descent(X, y, theta, alpha, epochs):
    m = len(y)
    J_history = []
    for i in range(epochs):
        h = sigmoid(np.dot(X, theta))
        h = np.clip(h, 0.000001, 0.999999)  
        gradient = (1/m) * np.dot(X.T, (h-y))
        theta -= alpha * gradient
        J_history.append(cost(X, y, theta))
    return theta, J_history

X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

m, n = X_train.shape
theta = np.zeros(n)
alpha = 0.01
epochs = 1000

theta, J_history = gradient_descent(X_train, y_train, theta, alpha, epochs)

y_pred_train = sigmoid(np.dot(X_train, theta))
y_pred_train = np.clip(y_pred_train, 0.000001, 0.999999)
y_pred_train = np.round(y_pred_train)

y_pred_test = sigmoid(np.dot(X_test, theta))
y_pred_test = np.clip(y_pred_test, 0.000001, 0.999999)
y_pred_test = np.round(y_pred_test)

acc_train = np.mean(y_pred_train == y_train)
acc_test = np.mean(y_pred_test == y_test)

print("Training accuracy:", acc_train)
print("Test accuracy:", acc_test)



Training accuracy: 0.905
Test accuracy: 0.92


We can apply Logistic Regression to a multi-class classification problem by splitting the problem into multiple binary classification problems. Yes it provided better results than in Question 4 as the accuracy increased by 0.01.

In [3]:
from google.colab import files
uploaded = files.upload()

Saving glass.csv to glass.csv
